# Reads a GCN AX dataframe (4D,3D or 2D) map this dataframe with the phase information from Zachs Excel-Sheet

In [1]:
# define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.notebook_imports import *
from pyforest import *
Console_and_file_logger('merge_data/gcn_05_2020', logging.INFO)

import SimpleITK as sitk
import random
from collections import Counter
from ipywidgets import interact

%matplotlib inline
plt.rcParams.update({'font.size': 30})
%reload_ext autoreload
%autoreload 2
pd.options.display.max_columns = None

from src.utils.utils_io import Console_and_file_logger, ensure_dir
from src.visualization.Visualize import plot_3d_vol, plot_4d_vol, plot_value_histogram, show_2D_or_3D
from src.data.Dataset import get_metadata_maybe, filter_4d_vol, copy_meta_and_save, create_3d_volumes_from_4d_files, describe_sitk, describe_volume, describe_path, get_phase, is_patient_in_df, get_extremas


search for root_dir and set working directory
Working directory set to: /mnt/data/git/cardio


2020-07-06 10:54:41,608 INFO -------------------- Start --------------------
2020-07-06 10:54:41,609 INFO Working directory: /mnt/data/git/cardio.
2020-07-06 10:54:41,609 INFO Log file: ./logs/merge_data/gcn_05_2020.log
2020-07-06 10:54:41,609 INFO Log level for console: INFO
Using TensorFlow backend.


# 1. Map phase and 4D dataframe

In [176]:
# Interativ dataframe chooser
# select the path to the phase excel-sheet
from ipyfilechooser import FileChooser
chooser = FileChooser('/mnt/data/datasets/cardio/GCN/', '')
display(chooser)

FileChooser(path='/mnt/data/datasets/cardio/GCN', filename='', show_hidden='False')

In [193]:
# read phase info
df_phase = pd.read_excel(chooser.selected)
print(df_phase.shape)
df_phase.head()

(101, 14)


,Patient Name,notes,bad outcome,study date,axial stack phases same as SA,numer of phases,ED,ES,wsx,upload complete,TH CHECK,YYYY,MM,DD
0,TET48V2Z,NaN,Y,2008-01-10 00:00:00,N,20,19,8,NaN,NaN,NaN,2008,1,10
1,0HQQW4ZN,NaN,Y,2007-05-23 00:00:00,Y,25,25,11,NaN,NaN,NaN,2007,5,23
2,2CW0G1A5,NaN,Y,2006-04-06 00:00:00,Y,25,25,11,NaN,NaN,NaN,2006,4,6
3,2WHFP73X,NaN,Y,2007-03-28 00:00:00,Y,25,21,11,NaN,NaN,NaN,2007,3,28
4,3E90836W,NaN,Y,2006-05-23 00:00:00,Y,30,28,12,NaN,NaN,NaN,2006,5,23


In [178]:
# Interativ dataframe chooser
# select the path to the dicom dataframe
from ipyfilechooser import FileChooser
df_chooser = FileChooser(os.path.join(os.getcwd(),'data/raw/gcn_05_2020_ax_sax_86'), '')
display(df_chooser)

FileChooser(path='/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_sax_86', filename='', show_hidden='False')

In [194]:
# read 4D dataframe
df_4d = pd.read_csv(df_chooser.selected)
print(df_4d.shape)
df_4d.head()

(86, 5)


,patient,x_path,y_path,timesteps,img_shape
0,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(10, 24)","(2, 24, 256, 256)"
1,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(0, 7)","(2, 16, 256, 256)"
2,0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(6, 24)","(2, 20, 288, 288)"
3,11YU3CUF,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(8, 23)","(2, 20, 288, 288)"
4,13JLP3HN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(12, 24)","(2, 22, 352, 352)"


In [195]:
df_4d['patient'] = df_4d['patient'].str.upper()

In [196]:
# extract the date from the filenames
temp = df_4d.x_path[0]
def extract_date(row):
    """
    helper to extract the date as tuple from the filename
    """
    date = os.path.basename(row['x_path']).split('_')[1].split('-')
    return int(date[0]), int(date[1]), int(date[2])

df_4d['YYYY'],df_4d['MM'], df_4d['DD'] = zip(*df_4d.apply(extract_date, axis=1))
df_4d.head()

,patient,x_path,y_path,timesteps,img_shape,YYYY,MM,DD
0,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(10, 24)","(2, 24, 256, 256)",2007,5,23
1,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(0, 7)","(2, 16, 256, 256)",2005,6,27
2,0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(6, 24)","(2, 20, 288, 288)",2007,2,13
3,11YU3CUF,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(8, 23)","(2, 20, 288, 288)",2007,12,10
4,13JLP3HN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(12, 24)","(2, 22, 352, 352)",2007,5,22


In [197]:
# Check if this file has no dublicated patient ids
len(df_4d.patient.unique()) == len(df_4d)

True

## Drop rows in the phase excel sheet without a phase mapping
## Make sure the Patient ID is with upper letters

In [198]:
# minor cleaning of the df_meta

df_phase_cleaned = df_phase[df_phase['ED'].notnull()].copy()
df_phase_cleaned['Patient Name'] = df_phase_cleaned['Patient Name'].apply(lambda x: str(x).replace(' ', ''))
df_phase_cleaned['Patient Name'] = df_phase_cleaned['Patient Name'].str.upper()
print('Cleaned phase dataframe: {}'.format(df_phase_cleaned.shape))

df_4d['patient'] = df_4d['patient'].apply(lambda x: str(x).replace(' ', ''))
df_4d['patient'] = df_4d['patient'].str.upper()

print(df_phase_cleaned.shape)
df_phase_cleaned.head()


Cleaned phase dataframe: (101, 14)
(101, 14)


,Patient Name,notes,bad outcome,study date,axial stack phases same as SA,numer of phases,ED,ES,wsx,upload complete,TH CHECK,YYYY,MM,DD
0,TET48V2Z,NaN,Y,2008-01-10 00:00:00,N,20,19,8,NaN,NaN,NaN,2008,1,10
1,0HQQW4ZN,NaN,Y,2007-05-23 00:00:00,Y,25,25,11,NaN,NaN,NaN,2007,5,23
2,2CW0G1A5,NaN,Y,2006-04-06 00:00:00,Y,25,25,11,NaN,NaN,NaN,2006,4,6
3,2WHFP73X,NaN,Y,2007-03-28 00:00:00,Y,25,21,11,NaN,NaN,NaN,2007,3,28
4,3E90836W,NaN,Y,2006-05-23 00:00:00,Y,30,28,12,NaN,NaN,NaN,2006,5,23


In [199]:
# extend the phase excel-sheet by the study date (YYYY, MM, DD)
# extract the date from the filenames
def extract_date_from_phaseformat(row):
    """
    helper to extract the date as tuple from the filename
    expects: a study date with day/month/year
    returns yyyy, mm, dd
    """
    try:
        date = row['study date'].year, row['study date'].month, row['study date'].day
        return int(date[0]), int(date[1]), int(date[2])
    except Exception as e:
        print(str(e))
        print(row['study date'])
        return 1900, 1, 1

df_phase_cleaned['YYYY'],df_phase_cleaned['MM'], df_phase_cleaned['DD'] = zip(*df_phase_cleaned.apply(extract_date_from_phaseformat, axis=1))
df_phase_cleaned.head()

'str' object has no attribute 'year'
? SAYS 1/1/1900
'str' object has no attribute 'year'
second 50 cases


,Patient Name,notes,bad outcome,study date,axial stack phases same as SA,numer of phases,ED,ES,wsx,upload complete,TH CHECK,YYYY,MM,DD
0,TET48V2Z,NaN,Y,2008-01-10 00:00:00,N,20,19,8,NaN,NaN,NaN,2008,1,10
1,0HQQW4ZN,NaN,Y,2007-05-23 00:00:00,Y,25,25,11,NaN,NaN,NaN,2007,5,23
2,2CW0G1A5,NaN,Y,2006-04-06 00:00:00,Y,25,25,11,NaN,NaN,NaN,2006,4,6
3,2WHFP73X,NaN,Y,2007-03-28 00:00:00,Y,25,21,11,NaN,NaN,NaN,2007,3,28
4,3E90836W,NaN,Y,2006-05-23 00:00:00,Y,30,28,12,NaN,NaN,NaN,2006,5,23


In [200]:
# extract the date from the filenames
def extract_date(row):
    """
    helper to extract the date as tuple from the filename
    """
    date = os.path.basename(row['x_path']).split('_')[1].split('-')
    return int(date[0]), int(date[1]), int(date[2])

df_4d['YYYY'],df_4d['MM'], df_4d['DD'] = zip(*df_4d.apply(extract_date, axis=1))
print(df_4d.shape)
df_4d.head(20)

(86, 8)


,patient,x_path,y_path,timesteps,img_shape,YYYY,MM,DD
0,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(10, 24)","(2, 24, 256, 256)",2007,5,23
1,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(0, 7)","(2, 16, 256, 256)",2005,6,27
2,0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(6, 24)","(2, 20, 288, 288)",2007,2,13
3,11YU3CUF,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(8, 23)","(2, 20, 288, 288)",2007,12,10
4,13JLP3HN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(12, 24)","(2, 22, 352, 352)",2007,5,22
5,1E4PF7MR,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(0, 13)","(2, 20, 256, 256)",2006,7,6
6,1E8MCV9L,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(0, 9)","(2, 28, 256, 256)",2005,12,15
7,1EMMCVKN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(9, 19)","(2, 20, 256, 256)",2006,6,29
8,1PFT9CA1,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(8, 23)","(2, 26, 256, 256)",2006,5,17
9,1ZZF5E55,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(7, 18)","(2, 24, 256, 256)",2006,10,1


In [201]:
# save the modified phase excel sheet from zach
# Overrides the previos file !!!
df_phase_cleaned.to_excel(df_phase_chooser.selected, index=False)

In [202]:
# compare the patient ids
patient_dicom, patient_phase = set(df_4d.patient.values), set(df_phase_cleaned['Patient Name'].values)
print('dicom patients: {}'.format(len(patient_dicom)))
print('phase patients: {}'.format(len(patient_phase)))
print('dicom patients, that are not in the excel-sheet: {}'.format(len(patient_dicom - patient_phase)))

dicom patients: 86
phase patients: 101
dicom patients, that are not in the excel-sheet: 0


In [203]:
# show the patients that are not in the phase excel sheet
patient_dicom - patient_phase

set()

## Inner join of the phase excel-sheet and the 4D dataframe

In [204]:
# merge the 4D dataframe with the cleaned phase info
# drop all rows without a short axis stack/ which are not contoured
df_merge = df_4d.merge(df_phase_cleaned, how='inner', left_on=['patient'], right_on=['Patient Name'])
print(df_merge.shape)
df_merge.head()

(86, 22)


,patient,x_path,y_path,timesteps,img_shape,YYYY_x,MM_x,DD_x,Patient Name,notes,bad outcome,study date,axial stack phases same as SA,numer of phases,ED,ES,wsx,upload complete,TH CHECK,YYYY_y,MM_y,DD_y
0,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(10, 24)","(2, 24, 256, 256)",2007,5,23,0HQQW4ZN,NaN,Y,2007-05-23 00:00:00,Y,25,25,11,NaN,NaN,NaN,2007,5,23
1,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(0, 7)","(2, 16, 256, 256)",2005,6,27,0PTV75MP,NaN,N,2005-06-27 00:00:00,Y,16,1,8,NaN,NaN,NaN,2005,6,27
2,0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(6, 24)","(2, 20, 288, 288)",2007,2,13,0RPELLU8,NaN,N,2007-02-13 00:00:00,Y,25,25,7,NaN,NaN,NaN,2007,2,13
3,11YU3CUF,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(8, 23)","(2, 20, 288, 288)",2007,12,10,11YU3CUF,NaN,N,2007-12-10 00:00:00,Y,25,24,9,NaN,NaN,NaN,2007,12,10
4,13JLP3HN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(12, 24)","(2, 22, 352, 352)",2007,5,22,13JLP3HN,NaN,N,2007-05-22 00:00:00,Y,25,25,13,NaN,NaN,NaN,2007,5,22


In [205]:
# check if we have a row with ED# == Nan
# this number reflects a dicom stack where we didn't find the phase
# patient 02Z1L14N, dicom exists, excel column exported == NAN
# patient RQ1JAJAU, dicom eists, excel column exported == NAN
# patient 778N33YG, dicom from 2005, phase from 2006
# patient RQ1JAJAU, dicom from 2007, phase from 2006
print(df_merge['ED'].isnull().sum())
df_merge[df_merge['ED'].isnull()]

0


,patient,x_path,y_path,timesteps,img_shape,YYYY_x,MM_x,DD_x,Patient Name,notes,bad outcome,study date,axial stack phases same as SA,numer of phases,ED,ES,wsx,upload complete,TH CHECK,YYYY_y,MM_y,DD_y


In [206]:
# check for double patient ids in the merged dataframe
# This means we mapped to rows from the timesteps xls
print(len(df_merge.patient.unique()))
from collections import Counter
c = Counter(df_merge.patient.values)
print(sorted(df_merge.patient.values, key=c.get, reverse=True)[:5])

86
['0HQQW4ZN', '0PTV75MP', '0RPELLU8', '11YU3CUF', '13JLP3HN']


In [207]:
df_merge.head()

,patient,x_path,y_path,timesteps,img_shape,YYYY_x,MM_x,DD_x,Patient Name,notes,bad outcome,study date,axial stack phases same as SA,numer of phases,ED,ES,wsx,upload complete,TH CHECK,YYYY_y,MM_y,DD_y
0,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(10, 24)","(2, 24, 256, 256)",2007,5,23,0HQQW4ZN,NaN,Y,2007-05-23 00:00:00,Y,25,25,11,NaN,NaN,NaN,2007,5,23
1,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(0, 7)","(2, 16, 256, 256)",2005,6,27,0PTV75MP,NaN,N,2005-06-27 00:00:00,Y,16,1,8,NaN,NaN,NaN,2005,6,27
2,0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(6, 24)","(2, 20, 288, 288)",2007,2,13,0RPELLU8,NaN,N,2007-02-13 00:00:00,Y,25,25,7,NaN,NaN,NaN,2007,2,13
3,11YU3CUF,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(8, 23)","(2, 20, 288, 288)",2007,12,10,11YU3CUF,NaN,N,2007-12-10 00:00:00,Y,25,24,9,NaN,NaN,NaN,2007,12,10
4,13JLP3HN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"(12, 24)","(2, 22, 352, 352)",2007,5,22,13JLP3HN,NaN,N,2007-05-22 00:00:00,Y,25,25,13,NaN,NaN,NaN,2007,5,22


## Find patients with labelled timesteps != 2

In [208]:
def get_number_of_contours(t_list):
    import ast
    return len(ast.literal_eval(t_list))
n_contours = df_merge.timesteps.apply(get_number_of_contours)


In [209]:
c = Counter(n_contours)

In [210]:
c

Counter({2: 86})

In [211]:
# save merged 4D dataframe
df_4d.to_csv('data/raw/gcn_05_2020_ax_sax_86/AX_4D_merged_with_phase.csv')

# 2. Map phase and 3D dataframe

In [212]:
# Interativ dataframe chooser
# select the path to the phase excel-sheet
from ipyfilechooser import FileChooser
df_phase_chooser = FileChooser('/mnt/data/datasets/cardio/GCN/', '')
display(df_phase_chooser)

FileChooser(path='/mnt/data/datasets/cardio/GCN', filename='', show_hidden='False')

In [213]:
# load both dataframes
# read phase info
df_phase = pd.read_excel(df_phase_chooser.selected)
print(df_phase.shape)
df_phase.head()

(101, 14)


,Patient Name,notes,bad outcome,study date,axial stack phases same as SA,numer of phases,ED,ES,wsx,upload complete,TH CHECK,YYYY,MM,DD
0,TET48V2Z,NaN,Y,2008-01-10 00:00:00,N,20,19,8,NaN,NaN,NaN,2008,1,10
1,0HQQW4ZN,NaN,Y,2007-05-23 00:00:00,Y,25,25,11,NaN,NaN,NaN,2007,5,23
2,2CW0G1A5,NaN,Y,2006-04-06 00:00:00,Y,25,25,11,NaN,NaN,NaN,2006,4,6
3,2WHFP73X,NaN,Y,2007-03-28 00:00:00,Y,25,21,11,NaN,NaN,NaN,2007,3,28
4,3E90836W,NaN,Y,2006-05-23 00:00:00,Y,30,28,12,NaN,NaN,NaN,2006,5,23


In [2]:
# Interativ dataframe chooser
# select the path to the 3D dataframe
from ipyfilechooser import FileChooser
df_4d_chooser = FileChooser(os.path.join(os.getcwd(),'data/raw/gcn_05_2020_ax_sax_86'), '')
display(df_4d_chooser)

FileChooser(path='/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_sax_86', filename='', show_hidden='False')

In [215]:
df_3d = pd.read_csv(df_4d_chooser.selected)
print(df_3d.shape)
df_3d.head()

(172, 3)


,patient,x_path,y_path
0,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...
1,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...
2,0000-0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...
3,0000-0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...
4,0000-0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...


## Extend the 3D dataframe by timestep and the date

In [216]:
# extract t from the filenames
def extract_t_from_filename(f_name):
    return int(os.path.basename(os.path.normpath(f_name)).split('__')[1].split('_')[0].replace('t',''))
df_3d['t'] = df_3d.x_path.apply(extract_t_from_filename)
df_3d.head()

,patient,x_path,y_path,t
0,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,10
1,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,24
2,0000-0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,0
3,0000-0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,7
4,0000-0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,24


In [ ]:
# extend by phase if already converted
def extract_phase_from_filename(f_name):
    return int(os.path.basename(os.path.normpath(f_name)).split('_')[-2])
df_3d['phase'] = df_3d.x_path.apply(extract_phase_from_filename)
df_3d.head()

In [217]:
# extract the date from the filenames
def extract_date(row):
    """
    helper to extract the date as tuple from the filename
    """
    date = os.path.basename(row['x_path']).split('_')[1].split('-')
    return int(date[0]), int(date[1]), int(date[2])

df_3d['YYYY'],df_3d['MM'], df_3d['DD'] = zip(*df_3d.apply(extract_date, axis=1))
print(df_3d.shape)
df_3d.head(20)

(172, 7)


,patient,x_path,y_path,t,YYYY,MM,DD
0,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,10,2007,5,23
1,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,24,2007,5,23
2,0000-0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,0,2005,6,27
3,0000-0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,7,2005,6,27
4,0000-0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,24,2007,2,13
5,0000-0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,6,2007,2,13
6,0000-11YU3CUF,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,23,2007,12,10
7,0000-11YU3CUF,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,8,2007,12,10
8,0000-13JLP3HN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,12,2007,5,22
9,0000-13JLP3HN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,24,2007,5,22


## Minor cleaning of both dataframes

In [218]:
# minor cleaning and dropping of the phase dataframe
# drop rows without Phase info
# clean empty spaces in ID column, transform to upper
df_phase_cleaned = df_phase[df_phase['ED'].notnull()].copy()
df_phase_cleaned['Patient Name'] = df_phase_cleaned['Patient Name'].apply(lambda x: str(x).replace(' ', ''))
df_phase_cleaned['Patient Name'] = df_phase_cleaned['Patient Name'].str.upper()
print('Cleaned phase dataframe: {}'.format(df_phase_cleaned.shape))

df_3d['patient'] = df_3d['patient'].apply(lambda x : x.split('-')[1] if '-' in x else x )
df_3d['patient'] = df_3d['patient'].apply(lambda x: str(x).replace(' ', ''))
df_3d['patient'] = df_3d['patient'].str.upper()
print('cleaned df 3D dataframe: {}'.format(df_3d.shape))
df_3d.head()

Cleaned phase dataframe: (101, 14)
cleaned df 3D dataframe: (172, 7)


,patient,x_path,y_path,t,YYYY,MM,DD
0,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,10,2007,5,23
1,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,24,2007,5,23
2,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,0,2005,6,27
3,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,7,2005,6,27
4,0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,24,2007,2,13


## Inner join of the 3D dataframe and the phase info

In [219]:
# compare the patient ids
patient_dicom, patient_phase = set(df_3d.patient.values), set(df_phase_cleaned['Patient Name'].values)
print('dicom patients: {}'.format(len(patient_dicom)))
print('phase patients: {}'.format(len(patient_phase)))
print('dicom patients, that are not in the excel-sheet: {}'.format(len(patient_dicom - patient_phase)))

dicom patients: 86
phase patients: 101
dicom patients, that are not in the excel-sheet: 0


In [220]:
# show the patients that are not in the phase excel sheet
print('dicom patients, that are not in the phase excel: {}'.format(patient_dicom - patient_phase))
print('phase patients, that are not in the dicom dataframe: {}'.format(patient_phase - patient_dicom))

dicom patients, that are not in the phase excel: set()
phase patients, that are not in the dicom dataframe: {'VLMLMLX4', 'D4PXE75F', 'D7LXF1KC', '4N4ZDJQ2', '8P8E7RN', '264V7142', 'CT8RR370', 'KW4MJ3XX', '2X1HM1YK', 'FV9UL3AC', '1Y8H8XLE', 'GL9QL330', '4A8MPQF4', '68UAYTYD', 'SECOND50CASES'}


In [221]:
df_merge = df_3d.merge(df_phase_cleaned[['Patient Name','YYYY', 'ED','ES']], how='inner', left_on=['patient'], right_on=['Patient Name'])
print(df_merge.shape)
print('dicom patients, missed due to the join: {}'.format(patient_dicom - set(df_merge['patient'].values)))
df_merge.head()

(172, 11)
dicom patients, missed due to the join: set()


,patient,x_path,y_path,t,YYYY_x,MM,DD,Patient Name,YYYY_y,ED,ES
0,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,10,2007,5,23,0HQQW4ZN,2007,25,11
1,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,24,2007,5,23,0HQQW4ZN,2007,25,11
2,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,0,2005,6,27,0PTV75MP,2005,1,8
3,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,7,2005,6,27,0PTV75MP,2005,1,8
4,0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,24,2007,2,13,0RPELLU8,2007,25,7


## For each row find the matching phase

In [222]:
def get_phase(row):
    # different index start (0 or 1), need to increase the timestep index by 1
    phase = 'PHASE_NOT_MAPPED'
    try:
        t_norm = int(row.t + 1)
        if int(row['ED']) == t_norm:
            phase = 'ED'
        if int(row['ES']) == t_norm:
            phase = 'ES'
        return phase
    except Exception as e:
        print(t_norm)
        print(str(e))
        return phase
df_merge['phase'] = df_merge.apply(get_phase, axis=1)
df_merge.head(20)


,patient,x_path,y_path,t,YYYY_x,MM,DD,Patient Name,YYYY_y,ED,ES,phase
0,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,10,2007,5,23,0HQQW4ZN,2007,25,11,ES
1,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,24,2007,5,23,0HQQW4ZN,2007,25,11,ED
2,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,0,2005,6,27,0PTV75MP,2005,1,8,ED
3,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,7,2005,6,27,0PTV75MP,2005,1,8,ES
4,0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,24,2007,2,13,0RPELLU8,2007,25,7,ED
5,0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,6,2007,2,13,0RPELLU8,2007,25,7,ES
6,11YU3CUF,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,23,2007,12,10,11YU3CUF,2007,24,9,ED
7,11YU3CUF,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,8,2007,12,10,11YU3CUF,2007,24,9,ES
8,13JLP3HN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,12,2007,5,22,13JLP3HN,2007,25,13,ES
9,13JLP3HN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,24,2007,5,22,13JLP3HN,2007,25,13,ED


## Check if we mapped a phase to each row

In [223]:
pd.value_counts(df_merge.phase)

ED    86
ES    86
Name: phase, dtype: int64

In [224]:
# show rows without phase info
df_merge[df_merge['phase']=='PHASE_NOT_MAPPED']

,patient,x_path,y_path,t,YYYY_x,MM,DD,Patient Name,YYYY_y,ED,ES,phase


## Fast Error tracing with searchable patient fields

In [225]:
@interact
def filter_df(search_str=''):
    search_str = search_str.upper()
    print(search_str)
    return df_merge[df_merge['patient'].str.contains(search_str)]

interactive(children=(Text(value='', description='search_str'), Output()), _dom_classes=('widget-interact',))

In [226]:
# save the 3D dataframe with phase info to disk
df_merge.to_csv('data/raw/gcn_05_2020_ax_sax_86/AX_3D_merged_with_phase.csv', index=False)